# Excess mortality in different countries during the COVID-19 pandemics

In [1]:
%matplotlib notebook

import numpy as np
import pandas as pd
import pylab as plt
import seaborn as sns
import matplotlib

from matplotlib.patches import Polygon
from sklearn.linear_model import LinearRegression
import datetime
import statsmodels.api as sm

In [2]:
# # map
# https://geopandas.org/docs/user_guide/mapping.html
# https://stackoverflow.com/questions/64873285/winkel-tripel-projection-in-geopandas

In [3]:
import math

def round_to_n(x, n):
    if x==0:
        return 0
    if np.isnan(x):
        return np.nan
    else:
        return round(x, -int(math.floor(math.log10(abs(x)))) + (n - 1))

In [4]:
df = pd.read_csv('https://github.com/akarlinsky/world_mortality/blob/main/world_mortality.csv?raw=true')

df_population = pd.read_csv('https://github.com/datasets/population/blob/master/data/population.csv?raw=true')

df_official = pd.read_csv('https://github.com/owid/covid-19-data/blob/master/public/data/owid-covid-data.csv?raw=true')

In [5]:
df_official_who = pd.read_csv('https://covid19.who.int/WHO-COVID-19-global-data.csv')
df_official_who.loc[df_official_who['Country'] == 'The United Kingdom', 'Country'] = 'United Kingdom'
df_official_who.loc[df_official_who['Country'] == 'United States of America', 'Country'] = 'United States'

# np.unique(df_official_who['Country'])

In [6]:
# Week of the first official covid19 death, to start the excess mortality calculation

def get_excess_begin(country, datapoints_per_year = 53):    
    mm = {'Bosnia':'Bosnia and Herzegovina'}
    country_lookup = country if country not in mm else mm[country]
    
    if datapoints_per_year > 12:
        d = df_official[(df_official['location']==country_lookup) 
                      & (df_official['total_deaths']>0)]['date'].values[0]
        beg = datetime.date(2020, int(d[5:7]), int(d[8:10])).isocalendar()[1]
    
    elif datapoints_per_year > 4 and datapoints_per_year <= 12:
        beg = 2  # March
        
    elif datapoints_per_year == 4:
        beg = 0 
        
    return beg

In [7]:
def predict(X, country, verbose=False):    
    # Fit regression model on pre-2020 data 
    ind = (X[:,0] < 2020) & (X[:,1]<53)
    m = np.max(X[ind,1])
    onehot = np.zeros((np.sum(ind), m))
    for i,k in enumerate(X[ind,1]):
        onehot[i,k-1] = 1
    predictors = np.concatenate((X[ind,:1], onehot), axis=1)
    reg = LinearRegression(fit_intercept=False).fit(predictors, X[ind,2])
    
    if verbose:
        est = sm.OLS(X[ind,2], predictors)
        est = est.fit()
        print(est.summary())
    
    # Compute 2020 baseline
    ind2 = X[:,0] == 2020
    predictors2020 = np.concatenate((np.ones((m,1))*2020, np.eye(m)), axis=1)
    baseline = reg.predict(predictors2020)
            
    # Week 53 usually does not have enough data, so we'll use 
    # the same baseline value as for week 52
    if np.max(X[:,1])==53:
        baseline = np.concatenate((baseline, [baseline[-1]]))
        
    # for Ireland week 1 is missing for unclear reason
    if np.min(X[ind,1])==2:
        baseline[0] = baseline[1]
    
    # Excess mortality
    ind2 = X[:,0] == 2020
    diff2020 = X[ind2,2] - baseline[X[ind2,1]-1]
    ind3 = X[:,0] == 2021
    diff2021 = X[ind3,2] - baseline[X[ind3,1]-1]
    excess_begin = get_excess_begin(country, baseline.size)
    total_excess = np.sum(diff2020[excess_begin:]) + np.sum(diff2021)
        
    # Manual fit for uncertainty computation
    if np.unique(X[ind,0]).size > 1:
        y = X[ind,2][:,np.newaxis]
        beta = np.linalg.pinv(predictors.T @ predictors) @ predictors.T @ y
        yhat = predictors @ beta
        sigma2 = np.sum((y-yhat)**2) / (y.size-predictors.shape[1])
        
        S = np.linalg.pinv(predictors.T @ predictors)
        w = np.zeros((m, 1))
        w[X[(X[:,0] == 2020) & (X[:,1] < 53),1]-1] = 1
        if np.max(X[:,1])==53:
            w[52-1] += 1
        w[:excess_begin] = 0
        w[X[ind3,1]-1] += 1
        
        if country=='Ireland':
            w[1] += w[0]
            w[0] = 0 
        
        p = 0
        for i,ww in enumerate(w):
            p += predictors2020[i] * ww
        p = p[:,np.newaxis]
                        
        predictive_var = sigma2 * np.sum(w) + sigma2 * p.T @ S @ p
        total_excess_std = np.sqrt(predictive_var)[0][0]
    else:
        total_excess_std = np.nan
       
    return baseline, total_excess, excess_begin, total_excess_std

In [8]:
countries = np.unique(df['country_name'])
print(countries.size, '\n')

allcountries = {}

for i, country in enumerate(countries):    
    print(country, end=', ' if i<countries.size-1 else '')
    if (i+1)%6 == 0:
        print('')
        
    assert(np.unique(df[(df['country_name']==country)]['time_unit']).size == 1)
    
    X = df[(df['country_name']==country)][['year','time','deaths']].values
    X = X[~np.isnan(X[:,2]),:]
    X = X.astype(int)
      
    baseline, total_excess, excess_begin, total_excess_std = predict(X, country)
        
    # https://en.wikipedia.org/wiki/Casualties_of_the_2020_Nagorno-Karabakh_war
    if country=='Armenia':
        total_excess -= 4000 # 3360
    if country=='Azerbaijan':
        total_excess -= 4000 # (2854+50)
        
    # August 2020 heatwave, weeks 32-34
    # 1539, 1583, 35, 660, 3688
    if country in ['Belgium','France','Luxembourg','Netherlands','Germany']:
        heatwave = np.sum(X[(X[:,0]==2020)&(X[:,1]>=32)&(X[:,1]<=34), 2])
        heatwave -= np.sum(baseline[32-1:34+1-1])
        total_excess -= heatwave
        print(f'*-{heatwave:.0f}*, ', end='')

    allcountries[country] = [X, baseline, total_excess, excess_begin, total_excess_std]

91 

Albania, Andorra, Armenia, Australia, Austria, Azerbaijan, 
Belarus, Belgium, *-1543*, Bolivia, Bosnia, Brazil, Bulgaria, 
Canada, Chile, Colombia, Costa Rica, Croatia, Cyprus, 
Czechia, Denmark, Ecuador, Egypt, El Salvador, Estonia, 
Finland, France, *-1594*, Georgia, Germany, *-3740*, Gibraltar, Greece, 
Greenland, Guatemala, Hong Kong, Hungary, Iceland, Iran, 
Ireland, Israel, Italy, Jamaica, Japan, Kazakhstan, 
Kosovo, Kyrgyzstan, Latvia, Liechtenstein, Lithuania, Luxembourg, 
*-31*, Macao, Malaysia, Malta, Mauritius, Mexico, Moldova, 
Mongolia, Montenegro, Netherlands, *-660*, New Zealand, Nicaragua, North Macedonia, 
Norway, Oman, Panama, Paraguay, Peru, Philippines, 
Poland, Portugal, Qatar, Romania, Russia, San Marino, 
Serbia, Singapore, Slovakia, Slovenia, South Africa, South Korea, 
Spain, Sweden, Switzerland, Taiwan, Tajikistan, Thailand, 
Transnistria, Tunisia, Ukraine, United Kingdom, United States, Uruguay, 
Uzbekistan

In [9]:
# heatwave

1537+1579+3680+35+660

7491

In [10]:
for c in allcountries:
    if allcountries[c][0][0][0] != 2015:
        print(f'{c:15} -- data start with {allcountries[c][0][0][0]}')

Chile           -- data start with 2016
Germany         -- data start with 2016
Ireland         -- data start with 2018
Peru            -- data start with 2017
South Africa    -- data start with 0
Transnistria    -- data start with 2016
Uruguay         -- data start with 0


In [11]:
# Export the baselines

with open('baselines.csv','w') as f:
    for c in allcountries:
        X, baseline, total_excess, excess_begin, total_excess_std = allcountries[c] 
        for i,b in enumerate(baseline):
            f.write(f'{c}, {i+1}, {b:.1f}\n')

In [12]:
# Population

mm = {'South Korea':'Korea, Rep.',
      'Slovakia':'Slovak Republic',
      'Russia':'Russian Federation',
      'Iran':'Iran, Islamic Rep.',
      'Czechia':'Czech Republic',
      'Egypt':'Egypt, Arab Rep.',
      'Hong Kong':'Hong Kong SAR, China',
      'Bosnia':'Bosnia and Herzegovina',
      'Kyrgyzstan':'Kyrgyz Republic',
      'Macao':'Macao SAR, China'}

pops = np.zeros(len(allcountries.keys()))
for i,m in enumerate(allcountries.keys()):
    # Russia's population should include Crimea because mortality figures do 
    # Ukraine's population should *not* include Crimea
    if m=='Russia':
        pops[i] = 146748590 # Rosstat, estimate for 1 Jan 2020    
    elif m=='Ukraine':
        pops[i] = 41762138 # Ukrstat 2020, according to Wikipedia
    elif m=='Taiwan':
        pops[i] = 23568378 # Wikipedia 
    elif m=='Transnistria':
        pops[i] = 467400 # Wikipedia
    elif m in mm:
        pops[i] = df_population[df_population['Country Name']==mm[m]]['Value'].values[-1]
    else:
        pops[i] = df_population[df_population['Country Name']==m]['Value'].values[-1]

In [13]:
# Daily reported numbers during the same time period

mm = {'Bosnia':'Bosnia and Herzegovina'}

official = np.zeros(len(allcountries.keys()))
officialnow = np.zeros(len(allcountries.keys()))

for i,m in enumerate(allcountries.keys()):
    howmany = np.max(allcountries[m][0][:,1])
    last = allcountries[m][0][-1][1]
    lastyear = allcountries[m][0][-1][0]
    
    if howmany > 12:
        d = f'{lastyear}-W{last:02}'
        r = datetime.datetime.strptime(d + '-0', "%G-W%V-%w")
        r = str(r)[:10]
    elif howmany > 4:
        if last < 12:
            r = f'{lastyear}-{last+1:02}-01'
        else:
            r = f'{lastyear+1}-01-01'
#         if m=='Brazil': # !!!!!!!!!!!!!TEMPORARY FIX!!!!!!!!!!!!!!!!!
#             r = f'{lastyear}-{last:02}-31' 
    elif howmany == 4:
        if m == 'Iran':
            # https://twitter.com/Mahan_Ghafari/status/1336338070249951236
            r = '2020-09-21'
        else:
            if last < 4:
                r = f'{lastyear}-{last*3+1:02}-01'
            else:
                r = f'{lastyear+1}-01-01'
    else:
        r = ''
        
    if m == 'Hong Kong':
        official[i] = 109 # December 1, 2020
        #https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv
        continue
        
    if m == 'Macao':
        official[i] = 0
        continue
        
    if m == 'Greenland':
        official[i] = 0
        continue
        
    if m == 'Transnistria':
        official[i] = 689 # From the Telegram channel https://t.me/novostipmrcom
        continue
    
    if m in mm:
        official[i] = df_official[(df_official['location']==mm[m])
                                  &(df_official['date']==r)]['total_deaths'].values[0]
        officialnow[i] = df_official[(df_official['location']==mm[m])]['total_deaths'].values[-1]
    else:
        official[i] = df_official[(df_official['location']==m)
                                  &(df_official['date']==r)]['total_deaths'].values[0]
        officialnow[i] = df_official[(df_official['location']==m)]['total_deaths'].values[-1]

In [14]:
print(np.nansum(official))

c = 0
for m in allcountries:
    c += allcountries[m][2]
print(c)

print(c - np.nansum(official))
print(c / np.nansum(official))

2246639.0
3446203.3492399496
1199564.3492399496
1.53393729443847


In [15]:
count = 0
for c in allcountries:
    z = np.abs(allcountries[c][2])/allcountries[c][4]
    if z>2 and allcountries[c][2]>0:
        count += 1
print(f'Statistically significant increase: {count}')      

count = 0
for c in allcountries:
    z = np.abs(allcountries[c][2])/allcountries[c][4]
    if z>2 and allcountries[c][2]<0:
        count += 1
print(f'Statistically significant dencrease: {count}')      

count = 0
for c in allcountries:
    z = np.abs(allcountries[c][2])/allcountries[c][4]
    if z<2:
        count += 1
print(f'Not statistically significant: {count}')      


count = 0
for c in allcountries:
    z = np.abs(allcountries[c][2])/allcountries[c][4]
    if np.isnan(z):
        count += 1
print(f'Impossible to assess significance: {count}')      

Statistically significant increase: 66
Statistically significant dencrease: 5
Not statistically significant: 18
Impossible to assess significance: 2


In [16]:
count = 0
for c in allcountries:
    if allcountries[c][1].size > 12:
        count += 1
print(f'Weekly data: {count}')

count = 0
for c in allcountries:
    if allcountries[c][1].size > 4 and allcountries[c][1].size <= 12:
        count += 1
print(f'Monthly data: {count}')

count = 0
for c in allcountries:
    if allcountries[c][1].size == 4:
        count += 1
print(f'Quarterly data: {count}')

Weekly data: 47
Monthly data: 42
Quarterly data: 2


## Figures

In [17]:
def stairs(x,y):
    xx = []
    yy = []
    for i in range(x.size):
        if i>0 and x[i]-x[i-1] > 1:
            skip = (x[i]-x[i-1]) - 1
            xx = xx + [np.nan, np.nan] * skip
            yy = yy + [np.nan, np.nan] * skip
        xx = xx + [x[i]-.5, x[i]+.5]
        yy = yy + [y[i], y[i]]
    return xx, yy

In [18]:
def percent_increase(country, zero_not_signif=False):
    X, baseline, total_excess, excess_begin, total_excess_std = allcountries[country] 
    d = total_excess/np.sum(baseline)*100
    
    if country=='Uruguay':
        d *= (baseline.size/53)
    if country=='El Salvador':
        d *= (baseline.size/12)
        
    if zero_not_signif and np.abs(total_excess)/total_excess_std < 2:
        d = 0 - ord(country[0])/100 - ord(country[1])/1000 # for alphabetical sorting
    return d

In [19]:
# ds = np.zeros(len(allcountries))
# for i,country in enumerate(allcountries.keys()):
#     ds[i] = percent_increase(country, zero_not_signif=True)
# ind = np.argsort(ds)[::-1]

# fig = plt.figure(figsize=(8*1.6,4.5*1.6))

# for i,country in enumerate(np.array(list(allcountries.keys()))[ind]):
#     plt.subplot(6,15,i+1)
    
#     X, baseline, total_excess, excess_begin, total_excess_std = allcountries[country] 

#     for year in np.arange(X[0,0],2020):
#         plt.plot(X[X[:,0]==year, 1], X[X[:,0]==year, 2], color='#aaaaaa', lw=1, clip_on=False)
#     plt.plot(X[X[:,0]==2020, 1], X[X[:,0]==2020, 2], color='r', lw=1.5, clip_on=False)
#     if np.sum(X[:,0]==2021)>1:
#         plt.plot(X[X[:,0]==2021, 1], X[X[:,0]==2021, 2], color='m', lw=1.5, clip_on=False)
#     elif np.sum(X[:,0]==2021)==1:
#         plt.plot(X[X[:,0]==2021, 1], X[X[:,0]==2021, 2], '.', color='m', markersize=3, clip_on=False)
#     plt.plot(np.arange(baseline.size)+1, baseline, color='k', lw=1, clip_on=False)

#     if i>=3: #np.max(X[:,-1]) < np.mean(baseline) * 2.75:
#         toplabel = .3
#     else:
#         toplabel = 1
        
#     if country in ['Belgium','France','Luxembourg','Netherlands','Germany','Armenia','Azerbaijan']:
#         star = '*'
#     else:
#         star = ''

#     fs = 8
#     thresh = 2
#     z = np.abs(total_excess)/total_excess_std
#     if np.isnan(z) or z > thresh:
#         plt.text(.0, .03, f'{round_to_n(total_excess,2):,.0f}'+star, transform=plt.gca().transAxes,
#                      color='r', fontsize=fs)
#         plt.text(1, .03, f'{percent_increase(country):.0f}%', transform=plt.gca().transAxes, 
#                  ha='right', color='#444444', fontsize=fs)
#         plt.text(1, toplabel, f'{round_to_n(total_excess/pops[ind][i]*100000,2):.0f}', transform=plt.gca().transAxes, 
#                  ha='right', va='top', color='k', fontsize=fs)
#         if total_excess>0:
#             plt.text(.0, toplabel, f'{total_excess/official[ind][i]:.1f}', transform=plt.gca().transAxes, 
#                      va='top', color='#0000aa', fontsize=fs)
#         else:
#             plt.text(.0, toplabel, '0', transform=plt.gca().transAxes, 
#                      va='top', color='#0000aa', fontsize=fs)
#     else:
#         plt.text(.0, .03, 'n.s.', transform=plt.gca().transAxes,
#             color='r', fontsize=fs)
        
#     poly1 = np.concatenate((X[X[:,0]==2020, 1][excess_begin:], X[X[:,0]==2020, 1][excess_begin:][::-1]))
#     poly2 = np.concatenate((X[X[:,0]==2020, 2][excess_begin:], baseline[X[X[:,0]==2020, 1]-1][excess_begin:][::-1]))
#     poly = np.concatenate((poly1[:,np.newaxis], poly2[:,np.newaxis]), axis=1)
#     poly = Polygon(poly, facecolor='r', edgecolor='r', alpha=.4, zorder=5)
#     plt.gca().add_patch(poly)
    
#     if np.sum(X[:,0]==2021)>1:
#         poly1 = np.concatenate((X[X[:,0]==2021, 1], X[X[:,0]==2021, 1][::-1]))
#         poly2 = np.concatenate((X[X[:,0]==2021, 2], baseline[X[X[:,0]==2021, 1]-1][::-1]))
#         poly = np.concatenate((poly1[:,np.newaxis], poly2[:,np.newaxis]), axis=1)
#         poly = Polygon(poly, facecolor='m', edgecolor='m', alpha=.4, zorder=5)
#         plt.gca().add_patch(poly)
    
#     plt.ylim([0,np.nanmax(X[:,-1])])
#     plt.xlim([1, baseline.size])
    
#     # For Uruguay:
#     if baseline.size > 12 and baseline.size < 50:
#         plt.xlim([1,53])
        
#     # For El Salvador:
#     if baseline.size > 4 and baseline.size < 12:
#         plt.xlim([1,12])
        
#     plt.xticks([])
#     plt.yticks([])
#     plt.title(country if country!='North Macedonia' else 'N. Macedonia', 
#               fontsize=9)
        
# sns.despine(left=True)
# plt.tight_layout(rect=(0,.05,1,1))

# fig.text(.01,.005, 
# 'Data: World Mortality Dataset, github.com/akarlinsky/world_mortality. '
# 'Analysis: github.com/dkobak/excess-mortality.\n'
# 'Excess mortality is computed relative to the baseline extrapolated from 2015–19. '
# 'Red number: excess mortality starting from the first officially reported covid19 death.\n'
# 'Gray: as a % of baseline yearly deaths. '
# 'Black: per 100,000 population. '
# 'Blue: ratio to the daily reported covid19 deaths over the same period. '
# f'Last update: {datetime.date.today():%b %d, %Y}', fontsize=8, va='bottom')

# fig.text(.99,.005, 'Ariel Karlinsky, @ArielKarlinsky\nDmitry Kobak, @hippopedoid', 
#          fontsize=8, ha='right', va='bottom')

# plt.savefig('img/all-countries-old.png', dpi=200)

In [20]:
# continents = {'East Europe':['Albania','Belarus','Bosnia','Bulgaria','Croatia','Cyprus','Czechia',
#                              'Estonia','Greece','Hungary','Kosovo','Latvia','Lithuania','Moldova',
#                              'Montenegro','North Macedonia','Poland','Romania', 'Russia','Serbia',
#                              'Slovakia', 'Slovenia','Ukraine',],
#               'West Europe':['Austria','Belgium','Denmark','Finland','France','Germany',
#                              'Iceland','Ireland','Italy','Liechtenstein','Luxembourg',
#                              'Malta','Netherlands','Norway','Portugal','San Marino',
#                              'Spain', 'Sweden', 'Switzerland','United Kingdom',],
#               'Asia':['Hong Kong','Japan','Macao', 'Malaysia','Mongolia','Philippines','Singapore',
#                       'South Korea','Taiwan','Thailand',],
#               'Oceania':['Australia','New Zealand',],
#               'Middle East Central Asia':['Armenia','Azerbaijan','Egypt','Georgia','Iran',
#                                           'Israel','Kazakhstan','Kyrgyzstan','Oman','Qatar',
#                                           'Tajikistan','Uzbekistan'],
#               'Africa':['Mauritius','South Africa','Tunisia',],
#               'Latin America':['Bolivia','Brazil','Chile','Colombia','Costa Rica','Ecuador',
#                                'Mexico','Peru','Uruguay',],
#               'North America':['Canada','United States',],}

# country2continent = {}
# for c in continents:
#     for cc in continents[c]:
#         country2continent[cc] = c
        
# continent_colors = {'East Europe':'#1b9e77',
#               'West Europe':'#d95f02',
#               'Asia':'#7570b3',
#               'Oceania':'#e7298a',
#               'Middle East Central Asia':'#66a61e',
#               'Africa':'#e6ab02',
#               'Latin America':'#a6761d',
#               'North America':'#666666'}

In [21]:
ds = np.zeros(len(allcountries))
for i,country in enumerate(allcountries.keys()):
    X, baseline, total_excess, excess_begin, total_excess_std = allcountries[country] 
    ds[i] = percent_increase(country, zero_not_signif=True)
ind = np.argsort(ds)[::-1]

fig = plt.figure(figsize=(8*1.75,4.5*1.75))

for i,country in enumerate(np.array(list(allcountries.keys()))[ind]):
    ax = plt.axes([.032+(i%16)*.061, .75-np.floor(i/16)*.145, .05, .1])
    
    col2020 = '#e41a1c' #continent_colors[country2continent[country]]  # 'r'
    col2021 = '#386cb0'# '#CA1F7B' #continent_colors[country2continent[country]]  # 'm'
    alpha = .5
    
    X, baseline, total_excess, excess_begin, total_excess_std = allcountries[country] 

    for year in np.arange(X[0,0],2020):
        plt.plot(X[X[:,0]==year, 1], X[X[:,0]==year, 2], color='#aaaaaa', lw=1, clip_on=False)
        
    if baseline.size < 50:
        xx1,yy1 = stairs(X[X[:,0]==2020, 1], X[X[:,0]==2020, 2])
        plt.plot(xx1, yy1, color=col2020, lw=1.5, clip_on=False)
    
        if np.sum(X[:,0]==2021)>0:
            xx3,yy3 = stairs(X[X[:,0]==2021, 1], X[X[:,0]==2021, 2])
            plt.plot(xx3, yy3, color=col2021, lw=1.5, clip_on=False)

        xx2,yy2 = stairs(np.arange(baseline.size)+1, baseline)
        plt.plot(xx2, yy2, color='k', lw=1, clip_on=False)
    else:
        xx1,yy1 = X[X[:,0]==2020, 1], X[X[:,0]==2020, 2]
        plt.plot(xx1, yy1, color=col2020, lw=1.5, clip_on=False)

        if np.sum(X[:,0]==2021)>1:
            xx3,yy3 = X[X[:,0]==2021, 1], X[X[:,0]==2021, 2]
            plt.plot(xx3, yy3, color=col2021, lw=1.5, clip_on=False)
        elif np.sum(X[:,0]==2021)==1:
            plt.plot(X[X[:,0]==2021, 1], X[X[:,0]==2021, 2], '.', color=col2021, markersize=3, clip_on=False)
        plt.plot(np.arange(baseline.size)+1, baseline, color='k', lw=1, clip_on=False)

        xx2,yy2 = np.arange(baseline.size)+1, baseline
        plt.plot(xx2, yy2, color='k', lw=1, clip_on=False)
        
    toplabel = .3
        
    if country in ['Belgium','France','Luxembourg','Netherlands','Germany','Armenia','Azerbaijan']:
        star = '*'
    else:
        star = ''

    fs = 8
    thresh = 2
    z = np.abs(total_excess)/total_excess_std
    if np.isnan(z) or z > thresh:
        plt.text(.0, .03, f'{round_to_n(total_excess,2):,.0f}'+star, transform=plt.gca().transAxes,
                     color='r', fontsize=fs)
        plt.text(1, .03, f'{percent_increase(country):.0f}%', transform=plt.gca().transAxes, 
                 ha='right', color='#444444', fontsize=fs)
        plt.text(1, toplabel, f'{round_to_n(total_excess/pops[ind][i]*100000,2):.0f}', transform=plt.gca().transAxes, 
                 ha='right', va='top', color='k', fontsize=fs)
        if total_excess>0:
            plt.text(.0, toplabel, f'{total_excess/official[ind][i]:.1f}', transform=plt.gca().transAxes, 
                     va='top', color='#0000aa', fontsize=fs)
        else:
            plt.text(.0, toplabel, '0', transform=plt.gca().transAxes, 
                     va='top', color='#0000aa', fontsize=fs)
    else:
        plt.text(.0, .03, 'n.s.', transform=plt.gca().transAxes,
            color='r', fontsize=fs)
        
    # Missing month fix
#     if country == 'Egypt':
#         xx1[-4] = 8.5
#         xx1[-3] = 9.5
#         yy1[-4] = yy2[-4]
#         yy1[-3] = yy2[-3]
        
    if baseline.size < 50:
        poly1 = np.concatenate((xx1[excess_begin*2:], xx1[excess_begin*2:][::-1]))
        poly2 = np.concatenate((yy1[excess_begin*2:], yy2[:len(yy1)][excess_begin*2:][::-1]))
    else:
        poly1 = np.concatenate((xx1[excess_begin:], xx1[excess_begin:][::-1]))
        poly2 = np.concatenate((yy1[excess_begin:], yy2[:len(yy1)][excess_begin:][::-1]))
    poly = np.concatenate((poly1[:,np.newaxis], poly2[:,np.newaxis]), axis=1)
    poly = Polygon(poly, facecolor=col2020, edgecolor=col2020, alpha=alpha, zorder=5, clip_on=False)
    plt.gca().add_patch(poly)
    
    if np.sum(X[:,0]==2021) > 1 or (baseline.size < 50 and np.sum(X[:,0]==2021) > 0):
        poly1 = np.concatenate((xx3, xx3[::-1]))
        poly2 = np.concatenate((yy3[:len(yy3)], yy2[:len(yy3)][::-1]))
        poly = np.concatenate((poly1[:,np.newaxis], poly2[:,np.newaxis]), axis=1)
        poly = Polygon(poly, facecolor=col2021, edgecolor=col2021, alpha=alpha, zorder=5, clip_on=False)
        plt.gca().add_patch(poly)
    
    plt.ylim([0,2*np.mean(baseline)])
    plt.xlim([.5, baseline.size+.5])
    plt.xticks([])
    
    # For Uruguay:
    if baseline.size > 12 and baseline.size < 50:
        plt.xlim([.5,53.5])
        
    # For El Salvador:
    if baseline.size > 4 and baseline.size < 12:
        plt.xlim([.5,12.5])
        
    plt.title(country if country!='North Macedonia' else 'N. Macedonia', 
              fontsize=9, y=.9)
        
    if i%16 > 0:
        plt.yticks([])
        sns.despine(left=True, ax=ax)
    else:
        plt.yticks([0,np.mean(baseline),np.mean(baseline)*2], ['0', '100%', '200%'], fontsize=7)
        sns.despine(ax=ax, offset={'left':3})
        plt.gca().set_yticklabels(['0', '100%', '200%'], fontsize=8)
        plt.gca().yaxis.set_tick_params(pad=1)

# plt.tight_layout(rect=(0,.05,1,1))

# from matplotlib import font_manager as fm
# myfont = fm.FontProperties(fname='fonts/static/EBGaramond-Regular.ttf')
# fig.text(.32,.93, 'Excess mortality during Covid-19', fontproperties=myfont, fontsize=28)

fig.text(.34, .915, 
'Data: World Mortality Dataset, github.com/akarlinsky/world_mortality. '
'Analysis: github.com/dkobak/excess-mortality.\n'
'Excess mortality is computed relative to the baseline extrapolated from 2015–19. '
'Red number: excess mortality starting from the first officially reported covid19 death.\n'
'Gray number: as a % of baseline annual deaths. '
'Black number: per 100,000 population. '
'Blue number: ratio to the daily reported covid19 deaths over the same period.\n'
'Red line and shading: 2020. Blue line and shading: 2021. Countries are sorted '
'by the excess deaths as a % of annual deaths.',
         fontsize=8, va='bottom')

# fig.text(.995,.005, f'Last update: {datetime.date.today():%b %d, %Y}\n\n'
#          'Ariel Karlinsky, @ArielKarlinsky\nDmitry Kobak, @hippopedoid', 
#          fontsize=8, ha='right', va='bottom')

fig.text(.995,.001, f'Last update: {datetime.date.today():%b %d, %Y}. '
         'Ariel Karlinsky, @ArielKarlinsky, Dmitry Kobak, @hippopedoid', 
         fontsize=8, ha='right', va='bottom')

plt.savefig('img/all-countries.png', dpi=200)

<IPython.core.display.Javascript object>

In [22]:
# # from matplotlib import rcParams
# # rcParams['font.sans-serif'] = ['Arial']

# fig = plt.figure(figsize=(7,9))

# for i,country in enumerate(allcountries.keys()):
#     plt.subplot(10,9,i+1)
    
#     X, baseline, total_excess, excess_begin, total_excess_std = allcountries[country] 

#     for year in np.arange(X[0,0],2020):
#         plt.plot(X[X[:,0]==year, 1], X[X[:,0]==year, 2], color='#aaaaaa', lw=1, clip_on=False)
#     plt.plot(X[X[:,0]==2020, 1], X[X[:,0]==2020, 2], color='r', lw=1.5, clip_on=False)
#     if np.sum(X[:,0]==2021)>1:
#         plt.plot(X[X[:,0]==2021, 1], X[X[:,0]==2021, 2], color='m', lw=1.5, clip_on=False)
#     elif np.sum(X[:,0]==2021)==1:
#         plt.plot(X[X[:,0]==2021, 1], X[X[:,0]==2021, 2], '.', color='m', markersize=3, clip_on=False)
#     plt.plot(np.arange(baseline.size)+1, baseline, color='k', lw=1, clip_on=False)

#     plt.plot(np.arange(baseline.size)+1, baseline, color='k', lw=1, clip_on=False)

#     if np.max(X[:,-1]) < np.mean(baseline) * 2.75:
#         toplabel = .37
#     else:
#         toplabel = 1
        
#     if country in ['Belgium','France','Luxembourg','Netherlands','Germany','Armenia','Azerbaijan']:
#         star = '*'
#     else:
#         star = ''

#     fs = 7
#     thresh = 2
#     z = np.abs(total_excess)/total_excess_std
#     if np.isnan(z) or z > thresh:
# #         if np.abs(total_excess)>100000:
# #             plt.text(.0, .03, f'{np.round(total_excess/1000)*1000:,.0f}'+star, transform=plt.gca().transAxes,
# #                      color='r', fontsize=fs)
# #         elif np.abs(total_excess)>1000:
# #             plt.text(.0, .03, f'{np.round(total_excess/100)*100:,.0f}'+star, transform=plt.gca().transAxes,
# #                      color='r', fontsize=fs)
# #         else:
#         plt.text(.0, .03, f'{round_to_n(total_excess,2):,.0f}'+star, transform=plt.gca().transAxes,
#                      color='r', fontsize=fs)
#         plt.text(1, .03, f'{percent_increase(country):.0f}%', transform=plt.gca().transAxes, 
#                  ha='right', color='#444444', fontsize=fs)
#         plt.text(1, toplabel, f'{round_to_n(total_excess/pops[i]*100000,2):.0f}', transform=plt.gca().transAxes, 
#                  ha='right', va='top', color='k', fontsize=fs)
#         if total_excess>0:
#             plt.text(.0, toplabel, f'{total_excess/official[i]:.1f}', transform=plt.gca().transAxes, 
#                      va='top', color='#0000aa', fontsize=fs)
#         else:
#             plt.text(.0, toplabel, '0', transform=plt.gca().transAxes, 
#                      va='top', color='#0000aa', fontsize=fs)
#     else:
#         plt.text(.0, .03, 'n.s.', transform=plt.gca().transAxes,
#             color='r', fontsize=fs)
        
#     poly1 = np.concatenate((X[X[:,0]==2020, 1][excess_begin:], X[X[:,0]==2020, 1][excess_begin:][::-1]))
#     poly2 = np.concatenate((X[X[:,0]==2020, 2][excess_begin:], baseline[X[X[:,0]==2020, 1]-1][excess_begin:][::-1]))
#     poly = np.concatenate((poly1[:,np.newaxis], poly2[:,np.newaxis]), axis=1)
#     poly = Polygon(poly, facecolor='r', edgecolor='r', alpha=.4, zorder=5)
#     plt.gca().add_patch(poly)
    
#     if np.sum(X[:,0]==2021)>1:
#         poly1 = np.concatenate((X[X[:,0]==2021, 1], X[X[:,0]==2021, 1][::-1]))
#         poly2 = np.concatenate((X[X[:,0]==2021, 2], baseline[X[X[:,0]==2021, 1]-1][::-1]))
#         poly = np.concatenate((poly1[:,np.newaxis], poly2[:,np.newaxis]), axis=1)
#         poly = Polygon(poly, facecolor='m', edgecolor='m', alpha=.4, zorder=5)
#         plt.gca().add_patch(poly)
    
#     plt.ylim([0,np.nanmax(X[:,-1])])
#     plt.xlim([1, baseline.size])
    
#     # For Uruguay:
#     if baseline.size > 12 and baseline.size < 50:
#         plt.xlim([1,53])
        
#     # For El Salvador:
#     if baseline.size > 4 and baseline.size < 12:
#         plt.xlim([1,12])
        
#     plt.xticks([])
#     plt.yticks([])
#     plt.title(country,# if country!='North Macedonia' else 'N. Macedonia', 
#               fontsize=8)
        
# sns.despine(left=True)
# plt.tight_layout()

# plt.savefig('img/all-countries-a4.pdf')

In [23]:
ds = np.zeros(len(allcountries))
for i,country in enumerate(allcountries.keys()):
    X, baseline, total_excess, excess_begin, total_excess_std = allcountries[country] 
    ds[i] = percent_increase(country, zero_not_signif=True)
ind = np.argsort(ds)[::-1]

fig = plt.figure(figsize=(7*1.5, 9*1.5))

for i,country in enumerate(np.array(list(allcountries.keys()))[ind]):
    ax = plt.axes([.05+(i%10)*.09, .85-np.floor(i/10)*.094, .08, .06])
    
    col2020 = '#e41a1c' 
    col2021 = '#386cb0'
    alpha = .5
    
    X, baseline, total_excess, excess_begin, total_excess_std = allcountries[country] 

    for year in np.arange(X[0,0],2020):
        plt.plot(X[X[:,0]==year, 1], X[X[:,0]==year, 2], color='#aaaaaa', lw=1, clip_on=False)
        
    if baseline.size < 50:
        xx1,yy1 = stairs(X[X[:,0]==2020, 1], X[X[:,0]==2020, 2])
        plt.plot(xx1, yy1, color=col2020, lw=1.5, clip_on=False)
    
        if np.sum(X[:,0]==2021)>0:
            xx3,yy3 = stairs(X[X[:,0]==2021, 1], X[X[:,0]==2021, 2])
            plt.plot(xx3, yy3, color=col2021, lw=1.5, clip_on=False)

        xx2,yy2 = stairs(np.arange(baseline.size)+1, baseline)
        plt.plot(xx2, yy2, color='k', lw=1, clip_on=False)
    else:
        xx1,yy1 = X[X[:,0]==2020, 1], X[X[:,0]==2020, 2]
        plt.plot(xx1, yy1, color=col2020, lw=1.5, clip_on=False)

        if np.sum(X[:,0]==2021)>1:
            xx3,yy3 = X[X[:,0]==2021, 1], X[X[:,0]==2021, 2]
            plt.plot(xx3, yy3, color=col2021, lw=1.5, clip_on=False)
        elif np.sum(X[:,0]==2021)==1:
            plt.plot(X[X[:,0]==2021, 1], X[X[:,0]==2021, 2], '.', color=col2021, markersize=3, clip_on=False)
        plt.plot(np.arange(baseline.size)+1, baseline, color='k', lw=1, clip_on=False)

        xx2,yy2 = np.arange(baseline.size)+1, baseline
        plt.plot(xx2, yy2, color='k', lw=1, clip_on=False)
        
    toplabel = .32
    bottomlabel = .03
        
    if country in ['Belgium','France','Luxembourg','Netherlands','Germany','Armenia','Azerbaijan']:
        star = '*'
    else:
        star = ''

    fs = 8
    thresh = 2
    z = np.abs(total_excess)/total_excess_std
    if np.isnan(z) or z > thresh:
        plt.text(.0, bottomlabel, f'{round_to_n(total_excess,2):,.0f}'+star, transform=plt.gca().transAxes,
                     color='r', fontsize=fs)
        plt.text(1, bottomlabel, f'{percent_increase(country):.0f}%', transform=plt.gca().transAxes, 
                 ha='right', color='#444444', fontsize=fs)
        plt.text(1, toplabel, f'{round_to_n(total_excess/pops[ind][i]*100000,2):.0f}', transform=plt.gca().transAxes, 
                 ha='right', va='top', color='k', fontsize=fs)
        if total_excess>0:
            plt.text(.0, toplabel, f'{total_excess/official[ind][i]:.1f}', transform=plt.gca().transAxes, 
                     va='top', color='#0000aa', fontsize=fs)
        else:
            plt.text(.0, toplabel, '0', transform=plt.gca().transAxes, 
                     va='top', color='#0000aa', fontsize=fs)
    else:
        plt.text(.0, bottomlabel, 'n.s.', transform=plt.gca().transAxes,
            color='r', fontsize=fs)
        
    if baseline.size < 50:
        poly1 = np.concatenate((xx1[excess_begin*2:], xx1[excess_begin*2:][::-1]))
        poly2 = np.concatenate((yy1[excess_begin*2:], yy2[:len(yy1)][excess_begin*2:][::-1]))
    else:
        poly1 = np.concatenate((xx1[excess_begin:], xx1[excess_begin:][::-1]))
        poly2 = np.concatenate((yy1[excess_begin:], yy2[:len(yy1)][excess_begin:][::-1]))
    poly = np.concatenate((poly1[:,np.newaxis], poly2[:,np.newaxis]), axis=1)
    poly = Polygon(poly, facecolor=col2020, edgecolor=col2020, alpha=alpha, zorder=5, clip_on=False)
    plt.gca().add_patch(poly)
    
    if np.sum(X[:,0]==2021) > 1 or (baseline.size < 50 and np.sum(X[:,0]==2021) > 0):
        poly1 = np.concatenate((xx3, xx3[::-1]))
        poly2 = np.concatenate((yy3[:len(yy3)], yy2[:len(yy3)][::-1]))
        poly = np.concatenate((poly1[:,np.newaxis], poly2[:,np.newaxis]), axis=1)
        poly = Polygon(poly, facecolor=col2021, edgecolor=col2021, alpha=alpha, zorder=5, clip_on=False)
        plt.gca().add_patch(poly)
    
    plt.ylim([0,2*np.mean(baseline)])
    plt.xlim([.5, baseline.size+.5])
    plt.xticks([])
    
    # For Uruguay:
    if baseline.size > 12 and baseline.size < 50:
        plt.xlim([.5,53.5])
        
    # For El Salvador:
    if baseline.size > 4 and baseline.size < 12:
        plt.xlim([.5,12.5])
        
    plt.title(country if country!='North Macedonia' else 'N. Macedonia', 
              fontsize=9, y=.9)
        
    if i%10 > 0:
        plt.yticks([])
        sns.despine(left=True, ax=ax)
    else:
        plt.yticks([0,np.mean(baseline),np.mean(baseline)*2], ['0', '100%', '200%'], fontsize=7)
        sns.despine(ax=ax, offset={'left':3})
        plt.gca().set_yticklabels(['0', '100%', '200%'], fontsize=8)
        plt.gca().yaxis.set_tick_params(pad=1)

plt.savefig('img/all-countries-a4-v2.pdf')

<IPython.core.display.Javascript object>

In [24]:
def until_date(country):
    howmany = np.max(allcountries[country][0][:,1])
    last = allcountries[country][0][-1][1]
    lastyear = allcountries[country][0][-1][0]

    if howmany>12:
        date = datetime.datetime.strptime(f'{lastyear}-W{last:02}-0', "%G-W%V-%w")
    elif howmany>4:
        if last<12:
            date = datetime.datetime.strptime(f'{lastyear}-{last+1:02}-01', "%Y-%m-%d")
        else:
            date = datetime.datetime.strptime(f'{lastyear+1}-01-01', "%Y-%m-%d")
        date -= datetime.timedelta(days=1)
    else:
        if last<4:
            date = datetime.datetime.strptime(f'{lastyear}-{last*3+1:02}-01', "%Y-%m-%d")
        else:
            date = datetime.datetime.strptime(f'{lastyear+1}-01-01', "%Y-%m-%d")
        date -= datetime.timedelta(days=1)
        
        if country=='Iran':
            if last>1:
                date = datetime.datetime.strptime(f'{lastyear}-{last*3:02}-21', "%Y-%m-%d")
            else:
                date = datetime.datetime.strptime(f'{lastyear-1}-12-21', "%Y-%m-%d")

    date = date.strftime('%b %d, %Y')
    
    if howmany>12:
        time_format = 'w'
    elif howmany>4:
        time_format = 'm'
    else:
        time_format = 'q'
    
    return date, time_format
    

print('Country                     Until  Type   Confirmed      Excess      std      t  Undercount  Per 100k   Increase')
print('----------------------------------------------------------------------------------------------------------------')

for i,c in enumerate(allcountries):
    until, time_format = until_date(c)
    thresh = 2
    
    z = np.abs(allcountries[c][2])/allcountries[c][4]
    if np.isnan(z) or z > thresh:
        undercount = f'        {round_to_n(np.max((0,allcountries[c][2]))/official[i],2):4.1f}' 
        per100k    = f'      {round_to_n(allcountries[c][2]/pops[i]*1e+5,2):4.0f}'
        increase   = f'        {percent_increase(c):2.0f}%'

    else:
        undercount = ''
        per100k    = ''
        increase   = ''
    
    print(f'{c:20} {until:15}  {time_format}     {round_to_n(official[i],2):7,.0f}'
          f'     {round_to_n(allcountries[c][2],2):7,.0f} ± {round_to_n(allcountries[c][4],2):6,.0f}' 
          f'  {z:5.1f}' + undercount + per100k + increase)

Country                     Until  Type   Confirmed      Excess      std      t  Undercount  Per 100k   Increase
----------------------------------------------------------------------------------------------------------------
Albania              Dec 31, 2020     m       1,200       6,000 ±    670    9.0         5.1       210        28%
Andorra              Dec 31, 2020     m          84          81 ±     26    3.1         1.0       110        25%
Armenia              Feb 28, 2021     m       3,200       7,000 ±    740    9.4         2.2       240        28%
Australia            Dec 27, 2020     w         910      -4,500 ±    880    5.1         0.0       -18        -3%
Austria              Apr 11, 2021     w       9,700       8,600 ±  1,200    7.1         0.9        97        10%
Azerbaijan           Dec 31, 2020     m       2,700      15,000 ±  1,200   12.5         5.7       150        27%
Belarus              Jun 30, 2020     m         400       5,700 ±    930    6.1        14.0     

In [25]:
for i,c in enumerate(allcountries):
    until, time_format = until_date(c)
    thresh = 2
    
    z = np.abs(allcountries[c][2])/allcountries[c][4]
    if np.isnan(z) or z > thresh:
        undercount = f'{round_to_n(np.max((0,allcountries[c][2]))/official[i],2):.1f}' 
        per100k    = f'${round_to_n(allcountries[c][2]/pops[i]*1e+5,2):.0f}$'
        increase   = f'${percent_increase(c):.0f}\%$'

    else:
        undercount = 'n.s.'
        per100k    = 'n.s.'
        increase   = 'n.s.'
        
    minus = '$-$' if allcountries[c][2]<0 else ''
    
    print(f'{c} & {until} & {time_format} & {round_to_n(official[i],2):,.0f} & '
          '' + minus + f'{round_to_n(np.abs(allcountries[c][2]),2):,.0f}'
          f'& $\pm\,${round_to_n(allcountries[c][4],2):,.0f} & ' 
          f'{z:.1f}' + ' & ' + undercount + ' & '  + per100k + ''
          ' & ' + increase + '\\\\')

Albania & Dec 31, 2020 & m & 1,200 & 6,000& $\pm\,$670 & 9.0 & 5.1 & $210$ & $28\%$\\
Andorra & Dec 31, 2020 & m & 84 & 81& $\pm\,$26 & 3.1 & 1.0 & $110$ & $25\%$\\
Armenia & Feb 28, 2021 & m & 3,200 & 7,000& $\pm\,$740 & 9.4 & 2.2 & $240$ & $28\%$\\
Australia & Dec 27, 2020 & w & 910 & $-$4,500& $\pm\,$880 & 5.1 & 0.0 & $-18$ & $-3\%$\\
Austria & Apr 11, 2021 & w & 9,700 & 8,600& $\pm\,$1,200 & 7.1 & 0.9 & $97$ & $10\%$\\
Azerbaijan & Dec 31, 2020 & m & 2,700 & 15,000& $\pm\,$1,200 & 12.5 & 5.7 & $150$ & $27\%$\\
Belarus & Jun 30, 2020 & m & 400 & 5,700& $\pm\,$930 & 6.1 & 14.0 & $60$ & $5\%$\\
Belgium & Apr 11, 2021 & w & 23,000 & 16,000& $\pm\,$1,600 & 10.0 & 0.7 & $140$ & $14\%$\\
Bolivia & Feb 28, 2021 & m & 12,000 & 29,000& $\pm\,$650 & 45.5 & 2.5 & $260$ & $56\%$\\
Bosnia & Dec 31, 2020 & m & 4,100 & 7,300& $\pm\,$810 & 9.0 & 1.8 & $220$ & $20\%$\\
Brazil & Mar 31, 2021 & m & 330,000 & 400,000& $\pm\,$13,000 & 31.0 & 1.2 & $190$ & $29\%$\\
Bulgaria & Apr 18, 2021 & w & 15,000 & 

In [26]:
# Export the table

with open('excess-mortality.csv','w') as f:
    f.write('Country,Data until,Data type,COVID-19 deaths,Excess deaths,'
            'Excess deaths std,t-statistic,Undercount ratio,Excess per 100k,'
            'Excess as % of annual baseline\n')
    
    for i,c in enumerate(allcountries):
        until, time_format = until_date(c)
         
        if official[i] > 0:
            undercount = np.max((0,allcountries[c][2]))/official[i]
        else:
            undercount = 0
        per100k    = allcountries[c][2]/pops[i]*1e+5
        increase   = percent_increase(c)
        
        z = np.abs(allcountries[c][2])/allcountries[c][4]
        if z < 2:
            undercount = np.nan
            
        f.write(f'{c},"{until}",{time_format},{official[i]:.0f},{allcountries[c][2]:.0f},'
                f'{allcountries[c][4]:.0f},{z:.1f},{undercount:.2f},{per100k:.1f},{increase:.1f}\n')

In [27]:
# # Placeholder to look stuff up

# country = 'Sweden'

# X,baseline,_,_,_ = allcountries[country] 
# plt.figure(figsize=(5,3))
# for year in np.arange(X[0,0],2020):
#     plt.plot(X[X[:,0]==year, 1], X[X[:,0]==year, 2], color='#aaaaaa', lw=1)
# plt.plot(X[X[:,0]==2020, 1], X[X[:,0]==2020, 2], color='r', lw=1.5, clip_on=False)
# plt.plot(X[X[:,0]==2020, 1][get_excess_begin('Sweden'):], 
#          X[X[:,0]==2020, 2][get_excess_begin('Sweden'):], '.', color='r', lw=1.5, clip_on=False)
# plt.plot(np.arange(baseline.size)+1, baseline, color='k', lw=1)

# for year in np.arange(X[0,0],2021):
#     print(np.sum(X[X[:,0]==year, 2]))

# # beg = excess_begin(country)
# # d = np.sum(diff[10:21])
# # d

# plt.title('Sweden')

# sns.despine()
# plt.tight_layout()
# plt.savefig('img/swe.png', dpi=100)

# # (X[X[:,0]==2020, 2] - baseline)

In [28]:
# X = allcountries['Belgium'][0]
# baseline = allcountries['Belgium'][1]

# plt.figure(figsize=(8*.75,4.5*.75))
# # a = 0
# for year in np.arange(2015,2020):
#     plt.plot(X[X[:,0]==year, 1], X[X[:,0]==year, 2], label=year)
# #     a+=X[(X[:,0]==year)&(X[:,1]<53), 2]
# # a/=5

# plt.plot(X[X[:,0]==2020, 1], X[X[:,0]==2020, 2], '.-', color='r', lw=2, label=2020)
# plt.plot(X[X[:,0]==2020, 1], baseline,'k' )    
# # plt.plot(X[X[:,0]==2020, 1], a, 'k--' )    
# plt.legend()

# np.sum(X[X[:,0]==2020, 2][31:34] - baseline[31:34])

In [29]:
# df = pd.read_csv('https://github.com/Financial-Times/coronavirus-excess-mortality-data/blob/master/data/ft_excess_deaths.csv?raw=true')
# X = df[(df['region']=='New York City')&(df['year']>=2015)][['year','week','deaths']].values

# plt.figure(figsize=(8*.75,4.5*.75))
# for year in np.arange(2015,2020):
#     plt.plot(X[X[:,0]==year, 1], X[X[:,0]==year, 2], color='#aaaaaa', lw=1)
# plt.plot(X[X[:,0]==2020, 1], X[X[:,0]==2020, 2], '.-', color='r', lw=1.5, clip_on=False)
# plt.xlim([1,53])
# plt.ylim([0,8000])
# plt.title('New York City')
# plt.xlabel('Week')
# plt.ylabel('Deaths per week')
# sns.despine()
# plt.tight_layout()

# print(np.sum(X[X[:,0]==2020, 2][10:25]-X[X[:,0]==2019, 2][10:25]))

# plt.savefig('img/nyc.png', dpi=100)

In [30]:
# X = allcountries['Ireland'][0]
# baseline = allcountries['Ireland'][1]

# plt.figure(figsize=(8*.75,4.5*.75))
# for year in np.arange(2015,2020):
#     plt.plot(X[X[:,0]==year, 1], X[X[:,0]==year, 2], label=year)

# plt.plot(X[X[:,0]==2020, 1], X[X[:,0]==2020, 2], '.-', color='r', lw=2, label=2020)
# plt.plot(X[X[:,0]==2021, 1], X[X[:,0]==2021, 2], '.-', color='m', lw=2, label=2020)
# plt.plot(X[X[:,0]==2020, 1], baseline,'k' )    
# plt.legend()

# print(np.sum(X[X[:,0]==2020, 2][11:21] - baseline[11:21]))
# print(np.sum(X[X[:,0]==2020, 2][11:] - baseline[11:]))
# print(np.sum(X[X[:,0]==2021, 2][:3] - baseline[:3]))

In [31]:
# X = allcountries['Italy'][0]
# baseline = allcountries['Italy'][1]

# plt.figure(figsize=(8*.75,4.5*.75))
# for year in np.arange(2015,2020):
#     plt.plot(X[X[:,0]==year, 1], X[X[:,0]==year, 2], label=year)

# plt.plot(X[X[:,0]==2020, 1], X[X[:,0]==2020, 2], '.-', color='r', lw=2, label=2020)
# plt.plot(X[X[:,0]==2021, 1], X[X[:,0]==2021, 2], '.-', color='m', lw=2, label=2020)
# plt.plot(np.arange(baseline.size)+1, baseline,'k' )    
# plt.legend()

In [32]:
# X = allcountries['Sweden'][0]
# baseline = allcountries['Sweden'][1]

# plt.figure(figsize=(8*.75,4.5*.75))
# for year in np.arange(2015,2020):
#     plt.plot(X[X[:,0]==year, 1], X[X[:,0]==year, 2], label=year)

# plt.plot(X[X[:,0]==2020, 1], X[X[:,0]==2020, 2], '.-', color='r', lw=2, label=2020)
# plt.plot(X[X[:,0]==2021, 1], X[X[:,0]==2021, 2], '.-', color='m', lw=2, label=2021)
# plt.plot(np.arange(baseline.size)+1, baseline,'k' )    
# plt.legend()

# plt.ylabel('Weekly deaths')
# plt.ylim([0,2600])
# plt.title('Sweden')
# sns.despine()
# plt.tight_layout()
# plt.savefig('img/swe.png', dpi=100)

In [33]:
# X = allcountries['Bolivia'][0]
# baseline = allcountries['Bolivia'][1]

# plt.figure(figsize=(8*.75,4.5*.75))
# for year in np.arange(2015,2020):
#     plt.plot(X[X[:,0]==year, 1], X[X[:,0]==year, 2], label=year)

# plt.plot(X[X[:,0]==2020, 1], X[X[:,0]==2020, 2], '.-', color='r', lw=2, label=2020)
# plt.plot(X[X[:,0]==2021, 1], X[X[:,0]==2021, 2], '.-', color='m', lw=2, label=2021)
# plt.plot(np.arange(baseline.size)+1, baseline,'k' )    
# plt.legend()

# np.sum(X[X[:,0]==2020, 2][3:]) - np.sum(baseline[4:])

In [34]:
# X = allcountries['France'][0]
# baseline = allcountries['France'][1]

# plt.figure(figsize=(8*.75,4.5*.75))
# for year in np.arange(2015,2020):
#     plt.plot(X[X[:,0]==year, 1], X[X[:,0]==year, 2], label=year)

# plt.plot(X[X[:,0]==2020, 1], X[X[:,0]==2020, 2], '.-', color='r', lw=2, label=2020)
# plt.plot(X[X[:,0]==2021, 1], X[X[:,0]==2021, 2], '.-', color='m', lw=2, label=2021)
# plt.plot(np.arange(baseline.size)+1, baseline,'k' )    
# plt.legend()

In [35]:
# X = allcountries['Peru'][0]
# baseline = allcountries['Peru'][1]

# plt.figure(figsize=(8*.75,4.5*.75))
# for year in np.arange(2015,2020):
#     plt.plot(X[X[:,0]==year, 1], X[X[:,0]==year, 2], label=year)

# plt.plot(X[X[:,0]==2020, 1], X[X[:,0]==2020, 2], '.-', color='r', lw=2, label=2020)
# plt.plot(X[X[:,0]==2021, 1], X[X[:,0]==2021, 2], '.-', color='m', lw=2, label=2021)
# plt.plot(np.arange(baseline.size)+1, baseline,'k', label='model')    
# plt.legend()

# sns.despine()
# plt.tight_layout()
# plt.savefig('img/peru2.png',dpi=100)

In [ ]:
d = df_official[df_official['location']==country]['total_deaths'].values

plt.figure(figsize=(8, 4.5))

plt.subplot(221)

x = np.zeros(len(allcountries))
y = np.zeros(len(allcountries))
z = np.zeros(len(allcountries))
for i,c in enumerate(allcountries):
    x[i] = allcountries[c][2] / 1000
    z[i] = allcountries[c][4] / 1000 * 1.96
    y[i] = x[i] / (official[i]+.001) * officialnow[i]
ind = np.argsort(x)[::-1][:10]
sc = plt.scatter(x[ind],-np.arange(10), zorder=3)
for i in range(10):
    plt.plot([x[ind][i]-z[ind][i], x[ind][i]+z[ind][i]], [-i,-i], c=sc.get_facecolors()[0], lw=1.5, zorder=1)
    plt.plot([x[ind][i]-z[ind][i], x[ind][i]-z[ind][i]], [-i-.2,-i+.2], c=sc.get_facecolors()[0], lw=1.5, zorder=1)
    plt.plot([x[ind][i]+z[ind][i], x[ind][i]+z[ind][i]], [-i-.2,-i+.2], c=sc.get_facecolors()[0], lw=1.5, zorder=1)
plt.scatter(y[ind],-np.arange(10), c='w', edgecolors='#666666', zorder=.5, s=15)
plt.yticks(-np.arange(10), np.array(list(allcountries.keys()))[ind])
xl = [0,700] # plt.xlim()
for i in range(10):
    plt.plot(xl,[-i,-i],lw=.5,color='#aaaaaa',zorder=0)
plt.xlim(xl)
plt.xticks([0,100,200,300,400,500,600,700])
plt.title('Excess deaths (thousands)', fontsize=10)

plt.text(y[ind][1],-8.7, f'Projection\nuntil now ({datetime.date.today():%b %d}):\n'
                         'official count now\ntimes the undercount', color='#666666', 
         fontsize=8, ha='center', bbox=dict(facecolor='w', edgecolor='#666666', lw=.75))
plt.plot([y[ind][1],y[ind][1]], [-8,-1], lw=.75, color='#666666',zorder=.1)
plt.text(x[ind][2],-4.5, 'Available data', color=sc.get_facecolors()[0], 
         fontsize=8, ha='center', bbox=dict(facecolor='w', edgecolor=sc.get_facecolors()[0], lw=.75))
plt.plot([x[ind][2],x[ind][2]], [-5,-2], lw=.75, color=sc.get_facecolors()[0],zorder=.1)

plt.subplot(224)

x = np.zeros(len(allcountries))
for i,c in enumerate(allcountries):
    x[i] = allcountries[c][2] / (official[i]+.001)
    z[i] = allcountries[c][4] / (official[i]+.001)
    if allcountries[c][2]/allcountries[c][4] < 2:
        x[i] = 0
ind = np.argsort(x)[::-1][:10]
sc = plt.scatter(x[ind],-np.arange(10), zorder=3)
for i in range(10):
    plt.plot([x[ind][i]-z[ind][i], x[ind][i]+z[ind][i]], [-i,-i], c=sc.get_facecolors()[0], lw=1.5, zorder=1)
    plt.plot([x[ind][i]-z[ind][i], x[ind][i]-z[ind][i]], [-i-.2,-i+.2], c=sc.get_facecolors()[0], lw=1.5, zorder=1)
    plt.plot([x[ind][i]+z[ind][i], x[ind][i]+z[ind][i]], [-i-.2,-i+.2], c=sc.get_facecolors()[0], lw=1.5, zorder=1)
plt.yticks(-np.arange(10), np.array(list(allcountries.keys()))[ind])
xl = [1,200] # plt.xlim()
for i in range(10):
    plt.plot(xl,[-i,-i],lw=.5,color='#aaaaaa',zorder=0)
plt.xlim(xl)
plt.title('Undercount ratio: excess / official', fontsize=10)
plt.xscale('log')

plt.subplot(222)

x = np.zeros(len(allcountries))
for i,c in enumerate(allcountries):
    x[i] = allcountries[c][2] / pops[i] * 1e+5
    y[i] = x[i] / (official[i]+.001)*officialnow[i]
    z[i] = allcountries[c][4] / pops[i] * 1e+5 * 1.96
    if pops[i] < 50000:
        x[i] = 0
ind = np.argsort(x)[::-1][:10]
sc = plt.scatter(x[ind],-np.arange(10), zorder=3, clip_on=False)
for i in range(10):
    plt.plot([x[ind][i]-z[ind][i], x[ind][i]+z[ind][i]], [-i,-i], c=sc.get_facecolors()[0], 
             lw=1.5, zorder=1, clip_on=False)
    plt.plot([x[ind][i]-z[ind][i], x[ind][i]-z[ind][i]], [-i-.2,-i+.2], c=sc.get_facecolors()[0], 
             lw=1.5, zorder=1, clip_on=False)
    plt.plot([x[ind][i]+z[ind][i], x[ind][i]+z[ind][i]], [-i-.2,-i+.2], c=sc.get_facecolors()[0], 
             lw=1.5, zorder=1, clip_on=False)
plt.scatter(y[ind],-np.arange(10), c='w', edgecolors='#666666', zorder=.5, s=15)
plt.yticks(-np.arange(10), np.array(list(allcountries.keys()))[ind])
xl = [0,500]
for i in range(10):
    plt.plot(xl,[-i,-i],lw=.5,color='#aaaaaa',zorder=0)
plt.xlim(xl)
plt.xticks([0,100,200,300,400,500])
plt.title('Excess deaths per 100,000 population', x=.45, fontsize=10)

plt.subplot(223)

x = np.zeros(len(allcountries))
for i,c in enumerate(allcountries):
    x[i] = percent_increase(c)
    z[i] = allcountries[c][4] / np.sum(allcountries[c][1]) * 100 * 1.96 ### DANGER ####
    y[i] = x[i] / (official[i]+.001)*officialnow[i]
    if pops[i] < 50000:
        x[i] = 0
ind = np.argsort(x)[::-1][:10]
sc = plt.scatter(x[ind],-np.arange(10), zorder=3)
for i in range(10):
    plt.plot([x[ind][i]-z[ind][i], x[ind][i]+z[ind][i]], [-i,-i], c=sc.get_facecolors()[0], 
             lw=1.5, zorder=1, clip_on=False)
    plt.plot([x[ind][i]-z[ind][i], x[ind][i]-z[ind][i]], [-i-.2,-i+.2], c=sc.get_facecolors()[0], 
             lw=1.5, zorder=1, clip_on=False)
    plt.plot([x[ind][i]+z[ind][i], x[ind][i]+z[ind][i]], [-i-.2,-i+.2], c=sc.get_facecolors()[0], 
             lw=1.5, zorder=1, clip_on=False)
plt.scatter(y[ind],-np.arange(10), c='w', edgecolors='#666666', zorder=.5, s=15)
plt.yticks(-np.arange(10), np.array(list(allcountries.keys()))[ind])
xl = [0,130] # plt.xlim()
for i in range(10):
    plt.plot(xl,[-i,-i],lw=.5,color='#aaaaaa',zorder=0)
plt.xlim(xl)
plt.xticks([0,10,20,30,40,50,60,70,80,90,100,110,120,130],
           ['0%','','','30%','','','60%','','','90%','','','120%',''])
plt.title('Excess deaths as % of annual deaths', fontsize=10)

sns.despine()
plt.tight_layout()

plt.savefig('img/leaderboard.png', dpi=200)

<IPython.core.display.Javascript object>

In [ ]:
d = df_official[df_official['location']==country]['total_deaths'].values

plt.figure(figsize=(8, 4.5))

plt.subplot(221)

x = np.zeros(len(allcountries))
y = np.zeros(len(allcountries))
z = np.zeros(len(allcountries))
for i,c in enumerate(allcountries):
    x[i] = allcountries[c][2] / 1000
    z[i] = allcountries[c][4] / 1000 * 1.96
    y[i] = x[i] / (official[i]+.001) * officialnow[i]
ind = np.argsort(x)[::-1][:10]
sc = plt.scatter(x[ind],-np.arange(10), zorder=3)
for i in range(10):
    plt.plot([x[ind][i]-z[ind][i], x[ind][i]+z[ind][i]], [-i,-i], c=sc.get_facecolors()[0], lw=1.5, zorder=1)
    plt.plot([x[ind][i]-z[ind][i], x[ind][i]-z[ind][i]], [-i-.2,-i+.2], c=sc.get_facecolors()[0], lw=1.5, zorder=1)
    plt.plot([x[ind][i]+z[ind][i], x[ind][i]+z[ind][i]], [-i-.2,-i+.2], c=sc.get_facecolors()[0], lw=1.5, zorder=1)
plt.yticks(-np.arange(10), np.array(list(allcountries.keys()))[ind])
xl = [0,700] # plt.xlim()
for i in range(10):
    plt.plot(xl,[-i,-i],lw=.5,color='#aaaaaa',zorder=0)
plt.xlim(xl)
plt.xticks([0,100,200,300,400,500,600,700])
plt.title('Excess deaths (thousands)', fontsize=10)

plt.subplot(224)

x = np.zeros(len(allcountries))
for i,c in enumerate(allcountries):
    x[i] = allcountries[c][2] / (official[i]+.001)
    z[i] = allcountries[c][4] / (official[i]+.001)
    if allcountries[c][2]/allcountries[c][4] < 2:
        x[i] = 0
ind = np.argsort(x)[::-1][:10]
sc = plt.scatter(x[ind],-np.arange(10), zorder=3)
for i in range(10):
    plt.plot([x[ind][i]-z[ind][i], x[ind][i]+z[ind][i]], [-i,-i], c=sc.get_facecolors()[0], lw=1.5, zorder=1)
    plt.plot([x[ind][i]-z[ind][i], x[ind][i]-z[ind][i]], [-i-.2,-i+.2], c=sc.get_facecolors()[0], lw=1.5, zorder=1)
    plt.plot([x[ind][i]+z[ind][i], x[ind][i]+z[ind][i]], [-i-.2,-i+.2], c=sc.get_facecolors()[0], lw=1.5, zorder=1)
plt.yticks(-np.arange(10), np.array(list(allcountries.keys()))[ind])
xl = [1,200] # plt.xlim()
for i in range(10):
    plt.plot(xl,[-i,-i],lw=.5,color='#aaaaaa',zorder=0)
plt.xlim(xl)
plt.title('Undercount ratio: excess / official', fontsize=10)
plt.xscale('log')

plt.subplot(222)

x = np.zeros(len(allcountries))
for i,c in enumerate(allcountries):
    x[i] = allcountries[c][2] / pops[i] * 1e+5
    y[i] = x[i] / (official[i]+.001)*officialnow[i]
    z[i] = allcountries[c][4] / pops[i] * 1e+5 * 1.96
    if pops[i] < 50000:
        x[i] = 0
ind = np.argsort(x)[::-1][:10]
sc = plt.scatter(x[ind],-np.arange(10), zorder=3, clip_on=False)
for i in range(10):
    plt.plot([x[ind][i]-z[ind][i], x[ind][i]+z[ind][i]], [-i,-i], c=sc.get_facecolors()[0], 
             lw=1.5, zorder=1, clip_on=False)
    plt.plot([x[ind][i]-z[ind][i], x[ind][i]-z[ind][i]], [-i-.2,-i+.2], c=sc.get_facecolors()[0], 
             lw=1.5, zorder=1, clip_on=False)
    plt.plot([x[ind][i]+z[ind][i], x[ind][i]+z[ind][i]], [-i-.2,-i+.2], c=sc.get_facecolors()[0], 
             lw=1.5, zorder=1, clip_on=False)
plt.yticks(-np.arange(10), np.array(list(allcountries.keys()))[ind])
xl = [0,500]
for i in range(10):
    plt.plot(xl,[-i,-i],lw=.5,color='#aaaaaa',zorder=0)
plt.xlim(xl)
plt.xticks([0,100,200,300,400,500])
plt.title('Excess deaths per 100,000 population', x=.45, fontsize=10)

plt.subplot(223)

x = np.zeros(len(allcountries))
for i,c in enumerate(allcountries):
    x[i] = percent_increase(c)
    z[i] = allcountries[c][4] / np.sum(allcountries[c][1]) * 100 * 1.96 ### DANGER ####
    y[i] = x[i] / (official[i]+.001)*officialnow[i]
    if pops[i] < 50000:
        x[i] = 0
ind = np.argsort(x)[::-1][:10]
sc = plt.scatter(x[ind],-np.arange(10), zorder=3)
for i in range(10):
    plt.plot([x[ind][i]-z[ind][i], x[ind][i]+z[ind][i]], [-i,-i], c=sc.get_facecolors()[0], 
             lw=1.5, zorder=1, clip_on=False)
    plt.plot([x[ind][i]-z[ind][i], x[ind][i]-z[ind][i]], [-i-.2,-i+.2], c=sc.get_facecolors()[0],
             lw=1.5, zorder=1, clip_on=False)
    plt.plot([x[ind][i]+z[ind][i], x[ind][i]+z[ind][i]], [-i-.2,-i+.2], c=sc.get_facecolors()[0], 
             lw=1.5, zorder=1, clip_on=False)
plt.yticks(-np.arange(10), np.array(list(allcountries.keys()))[ind])
xl = [0,130] # plt.xlim()
for i in range(10):
    plt.plot(xl,[-i,-i],lw=.5,color='#aaaaaa',zorder=0)
plt.xlim(xl)
plt.xticks([0,10,20,30,40,50,60,70,80,90,100,110,120,130],
           ['0%','','','30%','','','60%','','','90%','','','120%',''])
plt.title('Excess deaths as % of annual deaths', fontsize=10)

sns.despine()
plt.tight_layout()

plt.savefig('img/leaderboard.pdf')

In [ ]:
# Until the end of 2020

x = np.zeros(len(allcountries))
y = np.zeros(len(allcountries))
z = np.zeros(len(allcountries))

for i,c in enumerate(allcountries):
    d = allcountries[c][0][allcountries[c][0][:,0]==2020,2]
    b = allcountries[c][1]
    if d.size == b.size:
        x[i] = np.sum(d-b) / pops[i] * 1e+5
        y[i] = x[i]
        
    if c=='Armenia':
        y[i] -= 4000 / pops[i] * 1e+5
    if c=='Azerbaijan':
        y[i] -= 4000 / pops[i] * 1e+5
    if c=='Belgium':
        y[i] -= 1539 / pops[i] * 1e+5
    
    z[i] = y[i]
    if pops[i] < 50000:
        z[i] = 0

# ind = np.argsort(x)[::-1][:20]
# for i in ind:
#     c = np.array(list(allcountries.keys()))[i]
#     print(f'{c:15} {x[i]:.0f}')
# print('')

# ind = np.argsort(y)[::-1][:20]
# for i in ind:
#     c = np.array(list(allcountries.keys()))[i]
#     print(f'{c:15} {y[i]:.0f}')
# print('')

print('====== 2020 alone (January--December) ======')
ind = np.argsort(z)[::-1][:20]
for i in ind:
    c = np.array(list(allcountries.keys()))[i]
    print(f'{c:15} {z[i]:.0f}')

In [ ]:
# Until the end of 2020

x = np.zeros(len(allcountries))
y = np.zeros(len(allcountries))
z = np.zeros(len(allcountries))
e = np.zeros(len(allcountries))

for i,c in enumerate(allcountries):
    d = allcountries[c][0][allcountries[c][0][:,0]==2020,2]
    b = allcountries[c][1]
    if d.size == b.size:
        x[i] = np.sum(d-b) / np.sum(b) * 100
        y[i] = x[i]
        e[i] = np.sum(d-b)
        
    if c=='Armenia':
        y[i] -= 4000 / np.sum(b) * 100
    if c=='Azerbaijan':
        y[i] -= 4000 / np.sum(b) * 100
    if c=='Belgium':
        y[i] -= 1539 / np.sum(b) * 100
    
    z[i] = y[i]
    if pops[i] < 50000:
        z[i] = 0

print('====== 2020 alone (January--December) ======')
ind = np.argsort(z)[::-1][:20]
for i in ind:
    c = np.array(list(allcountries.keys()))[i]
    print(f'{c:15} {z[i]:.0f}%   [{e[i]:.0f} excess]')
print('    Note: tiny countries excluded')

In [ ]:
c = 'Russia'
d = allcountries[c][0][allcountries[c][0][:,0]==2020,2]
b = allcountries[c][1]
print(np.sum(d))
print(np.sum(b))
print(np.sum(d-b))
print(np.sum(d-b)/np.sum(b))

## Extrapolation until today

In [ ]:
countries = np.unique(df_official['location'])
stoplist = ['World','Europe','North America','European Union','South America','Asia','Africa']
countries = np.array([c for c in countries if c not in stoplist])
deaths = np.zeros(countries.size)
for i,country in enumerate(countries):
    d = df_official[df_official['location']==country]['total_deaths'].values
    if np.sum(~np.isnan(d)) > 0:
        deaths[i] = np.nanmax(d)
deaths[np.isnan(deaths)]=0
ind = np.argsort(deaths)[::-1][:30]
countries = countries[ind]
deaths = deaths[ind]

df_mine = pd.read_csv('excess-mortality.csv', index_col=None)

undercounts = np.zeros(30) * np.nan
for i,c in enumerate(countries):
    if c in df_mine['Country'].values:
        undercounts[i] = df_mine[df_mine['Country']==c]['Undercount ratio'].values[0]
        
mm = {'South Korea':'Korea, Rep.',
      'Slovakia':'Slovak Republic',
      'Russia':'Russian Federation',
      'Iran':'Iran, Islamic Rep.',
      'Czechia':'Czech Republic',
      'Egypt':'Egypt, Arab Rep.',
      'Hong Kong':'Hong Kong SAR, China',
      'Bosnia':'Bosnia and Herzegovina',
      'Kyrgyzstan':'Kyrgyz Republic',
      'Macao':'Macao SAR, China'}        

populations = np.zeros(30) * np.nan
for i,c in enumerate(countries):
#     if c in df_population['Country Name'].values:
    populations[i] = df_population[df_population['Country Name']==(mm[c] if c in mm else c)]['Value'].values[-1] / 1e+6
        
    if c=='Russia':
        populations[i] = 146.8

In [ ]:
data = [[countries[i],deaths[i],undercounts[i],populations[i]] for i in range(countries.size)]

df = pd.DataFrame(data, columns=['Country', 'Covid19 deaths, now', 
                                 'Underreporting (excess deaths / covid19 deaths)', 
                                 'Population (mln)'])

df['Estimated deaths'] = np.maximum(df['Underreporting (excess deaths / covid19 deaths)'],1) * df['Covid19 deaths, now']
df['Estimated deaths per 100,000'] = df['Estimated deaths'] / df['Population (mln)'] / 10

In [ ]:
df

In [ ]:
from matplotlib import font_manager as fm
myfont = fm.FontProperties(fname='fonts/static/EBGaramond-Regular.ttf')

fig = plt.figure(figsize=(8, 4.5))

plt.barh(-np.arange(df.shape[0]), df['Covid19 deaths, now'], label='Official daily reported\ncovid19 deaths')
delta = df['Estimated deaths'] - df['Covid19 deaths, now']
delta[delta<0] = 0
plt.barh(-np.arange(df.shape[0]), delta, left=df['Covid19 deaths, now'] + 2000, color='#d52323',
         label='FORECASTED excess deaths\n(official deaths times\nthe undercount estimate)')
         
plt.yticks(-np.arange(df.shape[0]), [a for a in df['Country']], fontsize=9)
plt.xlabel('Cumulative covid19 deaths and FORECASTED excess deaths')

for i in np.where(delta==0)[0]:
    plt.barh(-i, 2000, left=np.array(df['Covid19 deaths, now'])[i]-2000, color='k')
for i in np.where(np.isnan(delta))[0]:
    plt.text(np.array(df['Covid19 deaths, now'])[i] + 5000, -i-.3, 'n/a', color='r', fontsize=8)
for i in range(df.shape[0]):
    if np.isnan(df['Estimated deaths'].values[i]):
        plt.text(750000, -i-.3, 
             f"{df['Covid19 deaths, now'].values[i]/df['Population (mln)'].values[i]/10:.0f}",
             color='#598fb4', ha='right', fontsize=8)
        continue
    plt.text(df['Estimated deaths'].values[i] + 5000, -i-.3, 
             f"{df['Underreporting (excess deaths / covid19 deaths)'].values[i]:.1f}",
             color='k', fontsize=8)
    plt.text(750000, -i-.3, 
             f"{df['Estimated deaths per 100,000'].values[i]:.0f}",
             color='#d52323', ha='right', fontsize=8)
    
plt.title(f'Daily reported covid19 deaths and FORECASTED excess deaths until {datetime.date.today():%b %d}', 
          fontproperties=myfont, fontsize=18, x=.45)

plt.gca().tick_params(axis='y', length=0)

plt.gca().annotate('Undercount:\nexcess mortality /\ndaily reported\ncovid19 mortality',
                   xy=(550000, -7), xycoords='data',
                   xytext=(350000, -17), textcoords='data',
                   arrowprops=dict(arrowstyle="->", connectionstyle="arc3"),
                   fontsize=8)

plt.text(760000,-14,'Deaths per 100,000 people', rotation=90, va='center', fontsize=8)

plt.ylim([-df.shape[0]+.3,.7])
# plt.xlim([0,370000])

sns.despine()
plt.tight_layout()

plt.legend(loc='lower right', fontsize=9)

fig.text(.99,.02, 'Dmitry Kobak\n@hippopedoid', size=8, ha='right')

plt.savefig('img/countries.png', dpi=200)

## Weekly excess and weekly official

In [ ]:
def comparison(country):
    X, baseline, total_excess, excess_begin, total_excess_std = allcountries[country] 
    excess2020 = X[X[:,0]==2020, 2] - baseline
    excess2021 = X[X[:,0]==2021, 2] - baseline[:np.sum(X[:,0]==2021)]
    excess = np.concatenate((excess2020, excess2021))

    officialweekly = np.zeros(53+52) * np.nan
    for w in range(1,54):
        r = datetime.datetime.strptime(f'{2020}-W{w:02}-1', "%G-W%V-%w") # Monday
        r = str(r)[:10]
    
        ind = (df_official['location']==country) & (df_official['date']==r)
        if np.any(np.array(ind)):
            officialweekly[w-1] = df_official[ind]['total_deaths'].values[0]

    for w in range(1,53):
        r = datetime.datetime.strptime(f'{2021}-W{w:02}-1', "%G-W%V-%w")
        r = str(r)[:10]
    
        ind = (df_official['location']==country) & (df_official['date']==r)
        if np.any(np.array(ind)):
            officialweekly[53 + w-1] = df_official[ind]['total_deaths'].values[0]    
    
    officialweekly[np.where(~np.isnan(officialweekly))[0][0]-1] = 0
    officialweekly = np.diff(officialweekly)
    
    if country=='Peru':
        officialweekly[29] = np.nan
        officialweekly[32] = np.nan
    
    return excess, officialweekly

In [ ]:
countries = ['Austria','Belgium','Colombia','France',
             'Mexico','Netherlands','Peru','Spain',
             'Sweden','Switzerland','United Kingdom','United States']

plt.figure(figsize=(10,6))

for i, country in enumerate(countries):
    plt.subplot(3,4,i+1)
    excess, officialweekly = comparison(country)

    plt.plot(officialweekly, 'k', label='Official', lw=1.5)
    plt.plot(excess, '#e41a1c', label='Excess', lw=1.5)
    
    x = excess
    y = officialweekly[:excess.size]
    ind = ~np.isnan(x) & ~np.isnan(y)
    r = np.corrcoef(x[ind], y[ind])[0,1]

    plt.xlim([0,80])
    plt.plot([0,80], [0,0], 'k', lw=.75)
    yl = plt.ylim()
    plt.plot([53.5,53.5], yl, 'k--', lw=.75)
    plt.ylim(yl)
    plt.title(f'{country}, r={r:.2f}')

    lab1 = ['Jan','','','Apr','','','Jul','','','Oct','','',
            'Jan','','','Apr','','','']
    lab2 = [''] * len(lab1)
    plt.xticks(np.concatenate((np.linspace(0,53.5,13),
                           np.linspace(53.5 + 52.5/12, 53.5 + 52.5/12 * 6, 6))),
           lab1 if i>=8 else lab2, 
               rotation=70)
    
    if i==0:
        plt.legend(fontsize=8)

    if i%4 == 0:
        plt.ylabel('Weekly deaths')

sns.despine()
plt.gcf().align_labels()
plt.tight_layout()
plt.savefig('img/tracking_jhu.png', dpi=200)

In [ ]:
df_official_who = pd.read_csv('https://covid19.who.int/WHO-COVID-19-global-data.csv')
df_official_who.loc[df_official_who['Country'] == 'The United Kingdom', 'Country'] = 'United Kingdom'
df_official_who.loc[df_official_who['Country'] == 'United States of America', 'Country'] = 'United States'

In [ ]:
def comparison_who(country):
    X, baseline, total_excess, excess_begin, total_excess_std = allcountries[country] 
    excess2020 = X[X[:,0]==2020, 2] - baseline
    excess2021 = X[X[:,0]==2021, 2] - baseline[:np.sum(X[:,0]==2021)]
    excess = np.concatenate((excess2020, excess2021))

    officialweekly = np.zeros(53+52) * np.nan
    for w in range(1,54):
        r = datetime.datetime.strptime(f'{2020}-W{w:02}-1', "%G-W%V-%w") # Monday
        r = str(r)[:10]
    
        ind = (df_official_who['Country']==country) & (df_official_who['Date_reported']==r)
        if np.any(np.array(ind)):
            officialweekly[w-1] = df_official_who[ind]['Cumulative_deaths'].values[0]

    for w in range(1,53):
        r = datetime.datetime.strptime(f'{2021}-W{w:02}-1', "%G-W%V-%w")
        r = str(r)[:10]
    
        ind = (df_official_who['Country']==country) & (df_official_who['Date_reported']==r)
        if np.any(np.array(ind)):
            officialweekly[53 + w-1] = df_official_who[ind]['Cumulative_deaths'].values[0]    
    
    officialweekly[np.where(~np.isnan(officialweekly))[0][0]-1] = 0
    officialweekly = np.diff(officialweekly)
    
    if country=='Peru':
        officialweekly[29] = np.nan
        officialweekly[32] = np.nan
    
    return excess, officialweekly

In [ ]:
countries = ['Austria','Belgium','Colombia','France',
             'Mexico','Netherlands','Peru','Spain',
             'Sweden','Switzerland','United Kingdom','United States']

plt.figure(figsize=(10,6))

for i, country in enumerate(countries):
    plt.subplot(3,4,i+1)
    excess, officialweekly = comparison_who(country)

    plt.plot(officialweekly, 'k', label='Official', lw=1.5)
    plt.plot(excess, '#e41a1c', label='Excess', lw=1.5)
    
    x = excess
    y = officialweekly[:excess.size]
    ind = ~np.isnan(x) & ~np.isnan(y)
    r = np.corrcoef(x[ind], y[ind])[0,1]

    plt.xlim([0,80])
    plt.plot([0,80], [0,0], 'k', lw=.75)
    yl = plt.ylim()
    plt.plot([53.5,53.5], yl, 'k--', lw=.75)
    plt.ylim(yl)
    plt.title(f'{country}, r={r:.2f}')

    lab1 = ['Jan','','','Apr','','','Jul','','','Oct','','',
            'Jan','','','Apr','','','']
    lab2 = [''] * len(lab1)
    plt.xticks(np.concatenate((np.linspace(0,53.5,13),
                           np.linspace(53.5 + 52.5/12, 53.5 + 52.5/12 * 6, 6))),
           lab1 if i>=8 else lab2, 
               rotation=70)
    
    if i==0:
        plt.legend(fontsize=8)

    if i%4 == 0:
        plt.ylabel('Weekly deaths')

sns.despine()
plt.gcf().align_labels()
plt.tight_layout()
plt.savefig('img/tracking_who.png', dpi=200)

In [ ]:
country = 'Germany'
excess, officialweekly = comparison_who(country)

plt.figure(figsize=(5,3))

plt.plot(excess, 'r', label='Excess')
plt.plot(officialweekly, 'k', label='Official')

plt.xlim([0,80])
plt.plot([0,80], [0,0], 'k', lw=.75)
yl = plt.ylim()
plt.plot([53.5,53.5], yl, 'k--', lw=.75)
plt.ylim(yl)
plt.title(country)

plt.legend()

sns.despine()
plt.tight_layout()

In [ ]:
countries = ['Sweden', 'Spain', 'France', 'Belgium', 'Austria', 'Switzerland']

plt.figure(figsize=(8,4.5))

for i,country in enumerate(countries):

    excess, officialweekly_jhu = comparison(country)
    excess, officialweekly_who = comparison_who(country)

    plt.subplot(2,3,i+1)
    
    plt.plot(officialweekly_jhu, 'k', label='JHU', lw=1)
    plt.plot(officialweekly_who, 'r', label='WHO', lw=1)

    plt.xlim([0,80])
    plt.plot([0,80], [0,0], 'k', lw=.75)
    yl = plt.ylim()
    plt.plot([53.5,53.5], yl, 'k--', lw=.75)
    plt.ylim(yl)
    plt.title(country)

    if i==4:
        plt.legend()

sns.despine()
plt.tight_layout()
plt.savefig('img/who_vs_jhu.png', dpi=200)

In [ ]:
countries = ['Sweden', 'Spain', 'France', 'Belgium', 'Austria', 'Switzerland']#, 'Peru', 'Mexico']

plt.figure(figsize=(8,4.5))

for i,country in enumerate(countries):

    month = np.array([int(d[5:7]) for d in df_official_who['Date_reported'].values])
    officialweekly_who = df_official_who[(df_official_who['Country']==country)&(month>=3)]['New_deaths'].values
    month = np.array([int(d[5:7]) for d in df_official['date'].values])
    officialweekly_jhu = df_official[(df_official['location']==country)&(month>=3)]['new_deaths'].values
            
    plt.subplot(2,3,i+1)
    
    plt.plot(officialweekly_jhu, 'k', label='JHU', lw=1)
    plt.plot(officialweekly_who, 'r', label='WHO', lw=1)

    yl = plt.ylim()
    plt.title(country)

    if i==4:
        plt.legend()

sns.despine()
plt.tight_layout()
plt.savefig('img/who_vs_jhu_daily.png', dpi=200)